# 🤖 Multi-Tool Gemini Assistant using LangChain

In this notebook I built a **smart assistant** that decides which tool to use based on the user’s query — powered by **LangChain** and **Google Gemini**.

### 🎯 Goal
To practice **tool calling** and **agent reasoning** by creating a mini assistant that can:
- 🕒 Get current time of a city  
- 💱 Realtime currency converter
- 🌦️ Check weather  


In [12]:
# !pip install langchain langchain-google-genai python-dotenv langchain-core langchain-community
# !pip install requests

In [13]:
from langchain_core.tools import tool

In [48]:
# first tool
import requests
import os


@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city."""
    api_key = os.getenv("FREE_WEATHER_API_KEY")
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"
    response = requests.get(url)
    data = response.json()

    if data.get("error"):
        return f"Error: {data['error']['message']}"

    weather_desc = data["current"]["condition"]["text"]
    temp = data["current"]["temp_c"]
    humidity = data["current"]["humidity"]

    return f"City: {city}, Weather Conditon: {weather_desc}, Temperature {temp}°C, Humidity of {humidity}%."


In [49]:
# second tool

EXCHANGE_RATE_API_KEY = os.getenv("EXCHANGE_RATE_API_KEY")

@tool
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert from one currency to another based on real-time exchange rates.
    
    The model should:
    - Automatically convert country names (e.g., "India", "United States", "Japan") into their ISO 4217 currency codes
      (e.g., "INR", "USD", "JPY") before making the API call.
    - Accept either full country names or valid currency codes as input.
    """
    

    try:
        url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{from_currency.upper()}/{to_currency.upper()}"
        response = requests.get(url)    
        data = response.json()
        conversion_rate = data["conversion_rate"]

        converted_amount = amount * conversion_rate
        return f"{amount} {from_currency.upper()} is equal to {converted_amount:.2f} {to_currency.upper()}."
    
    except KeyError:
        print(KeyError)
        return "Error: Unable to retrieve conversion rate."

In [ ]:
# test the currency conversion tool

convert_currency.invoke({
    "amount": 100,
    "from_currency": "USD",
    "to_currency": "INR"
})


'100.0 USD is equal to 8877.58 INR.'

In [50]:
# third tool
from datetime import datetime

@tool
def get_time():
    """Get the current time in UTC."""
    return datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

llm_with_tools = llm.bind_tools([get_weather, convert_currency, get_time])

In [52]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Briefly guide me about the current weather of Faisalabad?")]
# messages = [HumanMessage(content="Convert 100 US Dollars to Indian Rupees.")]

ai_response = llm_with_tools.invoke(messages)

print(ai_response.additional_kwargs["function_call"])
messages.append(ai_response)

{'name': 'get_weather', 'arguments': '{"city": "Faisalabad"}'}


In [53]:
import json

function_calls = ai_response.additional_kwargs["function_call"]


if isinstance(function_calls, dict): # if llm returned a single function call
    function_calls = [function_calls]
    
    
for tool_call in function_calls:
    tool_name = tool_call["name"]
    
    tool_args = json.loads(tool_call["arguments"])
    
    if tool_name == "convert_currency":
        tool_result = convert_currency.invoke(tool_args)
    
    elif tool_name == "get_weather":
        tool_result = get_weather.invoke(tool_args)
    
    elif tool_name == "get_time":
        tool_result = get_time()
    
    messages.append(HumanMessage(tool_result))

In [54]:
print(messages)

[HumanMessage(content='Briefly guide me about the current weather of Faisalabad?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "Faisalabad"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f3fbf51e-a22b-4f71-a8de-f1ff11e608a5-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Faisalabad'}, 'id': '54736c2b-1e29-4c5d-bd1e-21cf70a73ce6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 148, 'output_tokens': 7, 'total_tokens': 155, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='City: Faisalabad, Weather Conditon: Clear, Temperature 26.4°C, Humidity of 19%.', additional_kwargs={}, response_metadata={})]


In [56]:
llm_with_tools.invoke(messages).content

'Okay, the current weather in Faisalabad is clear with a temperature of 26.4°C and 19% humidity.'